# NBA DFS Backtest Runner

Run backtests using the DailyBacktest and BacktestRunner classes from src.backtest module.

**Data Source**: Local SQLite database at `nba_dfs.db`

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import logging
from datetime import datetime, timedelta

sys.path.append('..')

from src.backtest.backtest import DailyBacktest, BacktestRunner, WalkForwardValidator
from src.data.collectors.tank01_client import Tank01Client
from src.data.collectors.local_data_client import LocalDataClient
from dotenv import load_dotenv

load_dotenv()

# Configure logging for Jupyter notebook with detailed output
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(name)-30s | %(levelname)-8s | %(message)s',
    datefmt='%H:%M:%S',
    force=True  # Force reconfiguration if already configured
)

# Set specific log levels for different modules
logging.getLogger('src.backtest.backtest').setLevel(logging.DEBUG)
logging.getLogger('src.evaluation.feature_builder').setLevel(logging.INFO)
logging.getLogger('src.evaluation.data_loader').setLevel(logging.INFO)

logger = logging.getLogger(__name__)


class MockStorage:
    """Mock storage for testing."""
    
    def __init__(self, base_path='../data'):
        self.base_path = base_path


print('=' * 80)
print('NOTEBOOK INITIALIZED')
print('=' * 80)
print(f'Logging level: {logging.getLevelName(logging.getLogger().level)}')
print(f'Backtest logging: {logging.getLevelName(logging.getLogger("src.backtest.backtest").level)}')
print('=' * 80)

NOTEBOOK INITIALIZED
Logging level: INFO
Backtest logging: DEBUG


## Initialize Components

Initialize LocalDataClient to read from local database and storage handler.

In [2]:
client = LocalDataClient()
storage = MockStorage(base_path='../data/tmp')

print('LocalDataClient initialized')
print(f'Database: {client.db_path}')
print(f'Data directory: {client.data_dir}')
print(f'Storage base path: {storage.base_path}')

LocalDataClient initialized
Database: C:\Users\antho\OneDrive\Documents\Repositories\delapan-fantasy\nba_dfs.db
Data directory: C:\Users\antho\OneDrive\Documents\Repositories\delapan-fantasy\data
Storage base path: ../data/tmp


## Option 1: Daily Backtest

Run backtest for single slate with data preparation and feature generation.

In [3]:
daily_backtest = DailyBacktest(
    api_client=client,
    storage=storage,
    seasons=['2023', '2024']  # Use valid seasons
)

print('=' * 80)
print('DailyBacktest initialized')
print('=' * 80)
print(f'API client: {type(client).__name__}')
print(f'Storage: {type(storage).__name__}')
print(f'Seasons: {daily_backtest.seasons}')
print('=' * 80)

DailyBacktest initialized
API client: LocalDataClient
Storage: MockStorage
Seasons: ['2023', '2024']


### Run Single Date Backtest

**Note:** 
- Database contains games from 2021-10-19 to 2024-04-30
- DFS salaries are loaded from database and merged with player logs
- The merged data includes salary information for lineup optimization

Use dates like:
- `20240412` (15 games)
- `20240414` (15 games) 
- `20240409` (14 games)

In [4]:

# Use a valid date from available data range (2021-10-19 to 2024-04-30)
game_date = '20250120'  # Date with 15 games

print(f'\n{"=" * 80}')
print(f'RUNNING BACKTEST FOR {game_date}')
print(f'{"=" * 80}\n')

result = daily_backtest.run_daily_backtest(
    game_date=game_date,
    lookback_days=90,
    train_model=True,
    evaluate_actuals=True,
    model_fn=None,
    optimizer_fn=None
)

print(f'\n{"=" * 80}')
print('BACKTEST COMPLETE')
print(f'{"=" * 80}')

if 'error' in result:
    print(f'\nERROR: {result["error"]}')
else:
    print(f'\n=== Summary ===')
    print(f'Game Date: {result.get("game_date")}')
    print(f'Teams: {len(result.get("teams", []))}')
    print(f'Players with features: {result.get("players_with_features", 0)}')
    
    if 'mape' in result:
        print(f'\n=== Metrics ===')
        print(f'MAPE:        {result["mape"]:.1f}%')
        print(f'RMSE:        {result["rmse"]:.2f} points')
        print(f'Correlation: {result["correlation"]:.3f}')
        print(f'Players evaluated: {result.get("num_evaluated", 0)}')
    
    if 'training_samples' in result:
        print(f'\n=== Training ===')
        print(f'Training samples: {result["training_samples"]}')
        print(f'Features: {result["num_features"]}')
        
    if 'merged_data' in result and not result['merged_data'].empty:
        merged_df = result['merged_data']
        print(f'\n=== Merged Data ===')
        print(f'Rows: {len(merged_df)}')
        print(f'Columns: {len(merged_df.columns)}')
        if 'salary' in merged_df.columns or 'dfs_salary' in merged_df.columns:
            salary_col = 'salary' if 'salary' in merged_df.columns else 'dfs_salary'
            print(f'Players with salaries: {merged_df[salary_col].notna().sum()}')
            print(f'\nSample merged data (first 3 players):')
            display_cols = ['longName', 'team', 'pts', 'reb', 'ast', salary_col]
            available_cols = [col for col in display_cols if col in merged_df.columns]
            print(merged_df[available_cols].head(3))

14:50:18 | src.backtest.backtest          | INFO     | Starting daily backtest for 20250120
14:50:18 | src.backtest.backtest          | INFO     | Preparing slate data for 20250120



RUNNING BACKTEST FOR 20250120


=== Running Backtest for 20250120 ===

[STEP 1] Checking games for 20250120...
  Found 8 games in database
  Games: 20250120_BOS@GS, 20250120_UTA@NO, 20250120_ATL@NY, 20250120_CHI@LAC, 20250120_MIN@MEM, 20250120_DAL@CHA, 20250120_DET@HOU, 20250120_PHO@CLE

  Sample game data:
          gameID away gameDate home teamIDAway teamIDHome
 20250120_BOS@GS  BOS 20250120   GS          2         10
 20250120_UTA@NO  UTA 20250120   NO         29         19
 20250120_ATL@NY  ATL 20250120   NY          1         20
20250120_CHI@LAC  CHI 20250120  LAC          5         13
20250120_MIN@MEM  MIN 20250120  MEM         18         15

[STEP 2] Preparing slate data...
Preparing data for 20250120
Seasons to collect: ['2023', '2024']
Teams playing: 16 teams, 16 total (home+away)
  LAC: 54 players
  CHI: 47 players
  DET: 71 players
  NO: 57 players
  DAL: 63 players
  GS: 47 players
  NY: 56 players
  HOU: 44 players
  ATL: 57 players
  PHO: 61 players
  CLE: 50 players


14:50:19 | src.backtest.backtest          | INFO     | Data preparation complete: 885 players found
14:50:19 | src.backtest.backtest          | INFO     | Verifying data completeness for 20250120


  CHA: 56 players
  MEM: 60 players
  UTA: 64 players
  BOS: 51 players
  MIN: 47 players
Total players: 885
Players with existing data: 885
Players needing data collection: 0

Loading player game logs from database...
Loaded 188 player game logs

Loading DFS salaries from database...
Loaded 22685 DFS salary records

Merging player logs with DFS salaries...
Merged DataFrame: 7350 rows, 39 columns
  Teams: ['LAC', 'CHI', 'DET', 'NO', 'DAL', 'GS', 'NY', 'HOU', 'ATL', 'PHO', 'CLE', 'CHA', 'MEM', 'UTA', 'BOS', 'MIN']
  Total players: 885

[STEP 3] Verifying data completeness...


14:50:19 | src.backtest.backtest          | INFO     | Verification complete: 885/885 players have data
14:50:19 | src.evaluation.feature_builder | INFO     | Building training features with strict temporal ordering
14:50:19 | src.evaluation.feature_builder | INFO     | Calculating DK fantasy points for training data


  Players with data: 885
  Players missing data: 0

[STEP 4] Loading historical training data (90 days lookback)...
  Date range: 20241022 to 20250120 (exclusive)
  Loaded 14741 player logs
  Unique players: 522
  Date range in data: 20241022 to 20250119

  Sample training data (first 5 rows):
gameDate    playerID      longName  pts  reb  ast mins
20241022        None          None None None None None
20241022        None          None None None None None
20241022 28968646399 Rui Hachimura   18    5    1   35
20241022 28828385129  Gabe Vincent    2    2    2   17
20241022 28858866027      Naz Reid   12    4    1   26

[STEP 5] Building training features...


14:50:19 | src.evaluation.feature_builder | INFO     | Calculating rolling features for each player
14:50:55 | src.evaluation.feature_builder | INFO     | Built training features: 11955 samples, 82 features
14:50:55 | src.backtest.backtest          | INFO     | Processing 885 players
14:50:55 | src.backtest.backtest          | INFO     | Preparing features for 885 players
14:50:55 | src.backtest.backtest          | DEBUG    | Loaded 274 game logs for player 28138596399 from database
14:50:55 | src.backtest.backtest          | DEBUG    | Calculating features for 28138596399 using 242 games
14:50:55 | src.backtest.backtest          | DEBUG    | Column fpts not found for player 28138596399
14:50:55 | src.backtest.backtest          | DEBUG    | Loaded 114 game logs for player 28998369499 from database
14:50:55 | src.backtest.backtest          | DEBUG    | Calculating features for 28998369499 using 88 games
14:50:55 | src.backtest.backtest          | DEBUG    | Column fpts not found for pla

  Training samples: 11955
  Features: 82
  Feature names (first 10): ['pts_avg_3', 'pts_std_3', 'pts_avg_5', 'pts_std_5', 'pts_avg_10', 'pts_std_10', 'pts_ewma', 'pts_min_5', 'pts_max_5', 'reb_avg_3']
  Target (y) range: min=-1.50, max=97.00, mean=21.83

[STEP 6] Model training...
  NOTE: Model implementation removed - implement your own model here
  Training data available: X_train (11955 samples, 82 features)
  Target data available: y_train (mean=21.83)

[STEP 7] Building slate features for 885 players...


14:50:55 | src.backtest.backtest          | DEBUG    | Loaded 142 game logs for player 94124209027 from database
14:50:55 | src.backtest.backtest          | DEBUG    | Calculating features for 94124209027 using 127 games
14:50:55 | src.backtest.backtest          | DEBUG    | Column fpts not found for player 94124209027
14:50:55 | src.backtest.backtest          | DEBUG    | Loaded 168 game logs for player 28988251252 from database
14:50:55 | src.backtest.backtest          | DEBUG    | Calculating features for 28988251252 using 131 games
14:50:55 | src.backtest.backtest          | DEBUG    | Column fpts not found for player 28988251252
14:50:55 | src.backtest.backtest          | DEBUG    | Loaded 32 game logs for player 28688877869 from database
14:50:55 | src.backtest.backtest          | DEBUG    | Calculating features for 28688877869 using 18 games
14:50:55 | src.backtest.backtest          | DEBUG    | Column fpts not found for player 28688877869
14:50:55 | src.backtest.backtest       

  Players with features: 852
  Feature columns: 47

  Sample player features (first 3):
   playerID        playerName team pos  games_played                                                                                                                                                                                                                         pts_avg_3                                                                                                                                                                  pts_avg_5                                                                                                                                                                 pts_avg_10
28138596399       Amir Coffey  LAC               242            32          NaN
33          NaN
34     9.666667
35     9.333333
36     8.000000
         ...   
269    0.000000
270    1.666667
271    1.666667
272    3.333333
273    3.333333
Name: pts, Length: 242, dtype: float64            3

In [ ]:
# Use a valid date from available data range (2021-10-19 to 2024-04-30)
game_date = '20240412'  # Date with 15 games

print(f'\n{"=" * 80}')
print(f'RUNNING BACKTEST FOR {game_date}')
print(f'{"=" * 80}\n')

result = daily_backtest.run_daily_backtest(
    game_date=game_date,
    lookback_days=90,
    train_model=True,
    evaluate_actuals=True,
    model_fn=None,
    optimizer_fn=None
)

print(f'\n{"=" * 80}')
print('BACKTEST COMPLETE')
print(f'{"=" * 80}')

if 'error' in result:
    print(f'\nERROR: {result["error"]}')
else:
    print(f'\n=== Summary ===')
    print(f'Game Date: {result.get("game_date")}')
    print(f'Teams: {len(result.get("teams", []))}')
    print(f'Players with features: {result.get("players_with_features", 0)}')
    
    if 'mape' in result:
        print(f'\n=== Metrics ===')
        print(f'MAPE:        {result["mape"]:.1f}%')
        print(f'RMSE:        {result["rmse"]:.2f} points')
        print(f'Correlation: {result["correlation"]:.3f}')
        print(f'Players evaluated: {result.get("num_evaluated", 0)}')
    
    if 'training_samples' in result:
        print(f'\n=== Training ===')
        print(f'Training samples: {result["training_samples"]}')
        print(f'Features: {result["num_features"]}')
        
    if 'merged_data' in result and not result['merged_data'].empty:
        merged_df = result['merged_data']
        print(f'\n=== Merged Data ===')
        print(f'Rows: {len(merged_df)}')
        print(f'Columns: {len(merged_df.columns)}')
        if 'salary' in merged_df.columns or 'dfs_salary' in merged_df.columns:
            salary_col = 'salary' if 'salary' in merged_df.columns else 'dfs_salary'
            print(f'Players with salaries: {merged_df[salary_col].notna().sum()}')
            print(f'\nSample merged data (first 3 players):')
            display_cols = ['longName', 'team', 'pts', 'reb', 'ast', salary_col]
            available_cols = [col for col in display_cols if col in merged_df.columns]
            print(merged_df[available_cols].head(3))

### Run Multi-Date Backtest

Run backtest across multiple consecutive dates with walk-forward validation.

In [ ]:
# Use valid date range from available data
start_date = datetime(2025, 1, 10)
end_date = datetime(2025, 1, 13)

game_dates = []
current_date = start_date
while current_date <= end_date:
    game_dates.append(current_date.strftime('%Y%m%d'))
    current_date += timedelta(days=1)

print(f'{"=" * 80}')
print(f'MULTI-DATE BACKTEST')
print(f'{"=" * 80}')
print(f'Date range: {game_dates[0]} to {game_dates[-1]}')
print(f'Total dates: {len(game_dates)}')
print(f'{"=" * 80}\n')

results_df = daily_backtest.run_multi_date_backtest(
    game_dates=game_dates,
    model_fn=None,
    optimizer_fn=None
)

print(f'\n{"=" * 80}')
print('MULTI-DATE BACKTEST COMPLETE')
print(f'{"=" * 80}')

if not results_df.empty:
    print(f'\nProcessed {len(results_df)} slates')
    
    if 'mape' in results_df.columns:
        print(f'\n=== Aggregate Metrics ===')
        print(f'Mean MAPE:   {results_df["mape"].mean():.1f}%')
        print(f'Median MAPE: {results_df["mape"].median():.1f}%')
        print(f'Std MAPE:    {results_df["mape"].std():.1f}%')
        print(f'Mean RMSE:   {results_df["rmse"].mean():.2f}')
        print(f'Mean Corr:   {results_df["correlation"].mean():.3f}')
        
        print(f'\n=== Daily Results ===')
        display_cols = ['game_date', 'players_with_features', 'num_evaluated', 'mape', 'rmse', 'correlation']
        available_cols = [col for col in display_cols if col in results_df.columns]
        print(results_df[available_cols].to_string(index=False))
else:
    print('\nNo results generated')

07:29:05 | src.backtest.backtest          | INFO     | Starting multi-date backtest for 4 dates
07:29:05 | src.backtest.backtest          | INFO     | Processing date 1/4: 20250110
07:29:05 | src.backtest.backtest          | INFO     | Starting daily backtest for 20250110
07:29:05 | src.backtest.backtest          | INFO     | Processing date 2/4: 20250111
07:29:05 | src.backtest.backtest          | INFO     | Starting daily backtest for 20250111
07:29:05 | src.backtest.backtest          | INFO     | Processing date 3/4: 20250112
07:29:05 | src.backtest.backtest          | INFO     | Starting daily backtest for 20250112
07:29:05 | src.backtest.backtest          | INFO     | Processing date 4/4: 20250113
07:29:05 | src.backtest.backtest          | INFO     | Starting daily backtest for 20250113
07:29:05 | src.backtest.backtest          | INFO     | Multi-date backtest complete: 4 successful, 0 failed


MULTI-DATE BACKTEST
Date range: 20250110 to 20250113
Total dates: 4


=== Running Multi-Date Backtest ===
Dates to process: 4


[1/4] Processing 20250110

=== Running Backtest for 20250110 ===

[STEP 1] Checking games for 20250110...
  Found 0 games in database
  Available date range: 2021-10-19 to 2024-04-30

[2/4] Processing 20250111

=== Running Backtest for 20250111 ===

[STEP 1] Checking games for 20250111...
  Found 0 games in database
  Available date range: 2021-10-19 to 2024-04-30

[3/4] Processing 20250112

=== Running Backtest for 20250112 ===

[STEP 1] Checking games for 20250112...
  Found 0 games in database
  Available date range: 2021-10-19 to 2024-04-30

[4/4] Processing 20250113

=== Running Backtest for 20250113 ===

[STEP 1] Checking games for 20250113...
  Found 0 games in database
  Available date range: 2021-10-19 to 2024-04-30

MULTI-DATE BACKTEST COMPLETE

No results generated


## Option 2: Walk-Forward Validation Backtest

Run walk-forward validation with train/test splits.

In [ ]:
summary = daily_backtest.get_summary()

print(f'\n{"=" * 80}')
print('SUMMARY STATISTICS')
print(f'{"=" * 80}')

if summary:
    for key, value in summary.items():
        print(f'{key}: {value}')
        
    if 'avg_players_per_slate' in summary:
        print(f'\nAverage processing:')
        print(f'  {summary.get("avg_players_per_slate", 0):.1f} players per slate')
        print(f'  {summary.get("data_completeness_rate", 0):.1f}% data completeness')
else:
    print('No summary statistics available (no backtests run yet)')
    
print(f'{"=" * 80}')

In [ ]:
# Visualize multi-date backtest results
import matplotlib.pyplot as plt

results_df = daily_backtest.get_results()

if not results_df.empty and 'mape' in results_df.columns:
    fig, axes = plt.subplots(3, 1, figsize=(14, 10))
    
    # Plot 1: MAPE over time
    axes[0].plot(range(len(results_df)), results_df['mape'], 'o-', linewidth=2, markersize=6, color='#2E86AB')
    axes[0].axhline(y=30, color='#A23B72', linestyle='--', linewidth=2, label='Target (30%)')
    axes[0].fill_between(range(len(results_df)), 0, 30, alpha=0.2, color='#06A77D')
    axes[0].set_ylabel('MAPE (%)', fontsize=12, fontweight='bold')
    axes[0].set_title('Projection Accuracy Over Time', fontsize=14, fontweight='bold')
    axes[0].legend(loc='upper right')
    axes[0].grid(True, alpha=0.3)
    axes[0].set_xticks(range(len(results_df)))
    axes[0].set_xticklabels(results_df['game_date'].values, rotation=45)
    
    # Plot 2: RMSE over time
    axes[1].plot(range(len(results_df)), results_df['rmse'], 's-', linewidth=2, markersize=6, color='#F18F01')
    axes[1].set_ylabel('RMSE (points)', fontsize=12, fontweight='bold')
    axes[1].set_title('Root Mean Squared Error Over Time', fontsize=14, fontweight='bold')
    axes[1].grid(True, alpha=0.3)
    axes[1].set_xticks(range(len(results_df)))
    axes[1].set_xticklabels(results_df['game_date'].values, rotation=45)
    
    # Plot 3: Correlation over time
    axes[2].plot(range(len(results_df)), results_df['correlation'], '^-', linewidth=2, markersize=6, color='#C73E1D')
    axes[2].axhline(y=0.5, color='gray', linestyle='--', linewidth=1, alpha=0.5)
    axes[2].set_ylabel('Correlation', fontsize=12, fontweight='bold')
    axes[2].set_xlabel('Date', fontsize=12, fontweight='bold')
    axes[2].set_title('Prediction Correlation Over Time', fontsize=14, fontweight='bold')
    axes[2].grid(True, alpha=0.3)
    axes[2].set_ylim(-0.1, 1.0)
    axes[2].set_xticks(range(len(results_df)))
    axes[2].set_xticklabels(results_df['game_date'].values, rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    print(f'\n{"=" * 80}')
    print('VISUALIZATION COMPLETE')
    print(f'{"=" * 80}')
    print(f'Plotted {len(results_df)} slates')
    print(f'Mean MAPE: {results_df["mape"].mean():.1f}%')
    print(f'{"=" * 80}')
else:
    print('No results available to plot. Run multi-date backtest first.')
    print(f'Current results: {len(results_df)} rows')

### Define Feature Columns

In [ ]:
feature_columns = [
    'mins', 'pts', 'reb', 'ast', 'stl', 'blk', 'TOV',
    'fga', 'fgm', 'FGP', 'tpa', 'tpm', 'TPP',
    'fta', 'ftm', 'FTP', 'PF'
]

target_column = 'fpts'

print(f'Feature columns: {len(feature_columns)}')
print(f'Target column: {target_column}')

Feature columns: 17
Target column: fpts


### Run Walk-Forward Backtest

In [ ]:
backtest_start = '20241201'
backtest_end = '20241231'

print(f'Running walk-forward backtest from {backtest_start} to {backtest_end}')

backtest_results = backtest_runner.run(
    start_date=backtest_start,
    end_date=backtest_end,
    feature_columns=feature_columns,
    target_column=target_column
)

print('\n=== Walk-Forward Backtest Results ===')
for key, value in backtest_results.items():
    print(f'{key}: {value}')

2025-10-06 05:51:11,860 - src.backtest.backtest - INFO - Starting BacktestRunner from 20241201 to 20241231
2025-10-06 05:51:11,887 - src.backtest.backtest - INFO - Loaded 4473 rows from database for date range 20241201 to 20241231
2025-10-06 05:51:11,888 - src.backtest.backtest - INFO - Loaded 4473 rows of data
2025-10-06 05:51:11,889 - src.backtest.backtest - INFO - Creating walk-forward splits: train_window=30, test_window=1, step_size=1


Running walk-forward backtest from 20241201 to 20241231


KeyError: 'date'

### View Detailed Fold Results

In [ ]:
fold_results = backtest_runner.get_results()

print('\n=== Fold-by-Fold Results ===')
print(fold_results[['fold', 'train_start', 'train_end', 'test_start', 'test_end', 'mape', 'rmse', 'correlation']])

### Plot MAPE Over Time

In [ ]:
import matplotlib.pyplot as plt

if not fold_results.empty:
    plt.figure(figsize=(12, 6))
    plt.plot(fold_results['fold'], fold_results['mape'], marker='o', label='MAPE')
    plt.axhline(y=30, color='r', linestyle='--', label='30% Target')
    plt.xlabel('Fold')
    plt.ylabel('MAPE')
    plt.title('Model MAPE Across Walk-Forward Folds')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print('No fold results to plot')

import matplotlib.pyplot as plt

fold_results = backtest_runner.get_results()

if not fold_results.empty and 'mape' in fold_results.columns:
    fig, axes = plt.subplots(2, 1, figsize=(14, 8))
    
    # Plot 1: MAPE over folds
    axes[0].plot(fold_results['fold'], fold_results['mape'], 'o-', linewidth=2, markersize=6)
    axes[0].axhline(y=30, color='r', linestyle='--', linewidth=2, label='30% Target')
    axes[0].fill_between(fold_results['fold'], 0, 30, alpha=0.2, color='green')
    axes[0].set_xlabel('Fold', fontsize=12)
    axes[0].set_ylabel('MAPE (%)', fontsize=12)
    axes[0].set_title('Model MAPE Across Walk-Forward Folds', fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Plot 2: RMSE over folds
    axes[1].plot(fold_results['fold'], fold_results['rmse'], 's-', linewidth=2, markersize=6, color='orange')
    axes[1].set_xlabel('Fold', fontsize=12)
    axes[1].set_ylabel('RMSE (points)', fontsize=12)
    axes[1].set_title('Model RMSE Across Walk-Forward Folds', fontsize=14, fontweight='bold')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f'\nPlotted {len(fold_results)} folds')
    print(f'Mean MAPE: {fold_results["mape"].mean():.2f}%')
    print(f'Mean RMSE: {fold_results["rmse"].mean():.2f}')
else:
    print('No fold results to plot')
    print(f'Available columns: {fold_results.columns.tolist() if not fold_results.empty else "None"}')

In [ ]:
print(f'\nTotal data queries: {client.get_request_count()}')
print(f'Data source: Local database at {client.db_path}')